# Ethiopia Mapping Section

This Python Jupyter file is to calculate and build out the requirements for the Ethiopian design. It could be possible to do this in Excel as well; but this way we have an ability to reference and redesign as per the changing requirements.

First Step is to collect the information; I have a Github account under my username (johnmeye) which i will reference from the file so that anyone who uses Conda/Jupyter will be able to get the files. For any challenges reach out to me on teams or by email (johnmeye@cisco.com)

## Second Revision

This is the second revision of this file as there was a business case change from Vodacom. As a result I'm rewriting this to make it clearer and smoother to calculate the required output. 

## Inputs

The following are the inputs to this file received from Vodacom.
1. Site specifications 
2. Site locations
2. Business case 
3. Consumption assumption of users

## Outputs 

The following are the expected outputs of this file:
1. BoM for sites in catagories of:

    a. POC1
    
    b. POC2
    
    c. POC3
    
    d. Access
    
    e. Peering
    
    
2. BoM's will be in the correct format for CCW upload to allow for quick creation of the total costs.

In [1]:
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#For the Map Plotting
import plotly
import plotly.express as px

#For the Inline display of figures.
from IPython.display import HTML
from IPython.display import IFrame

#For the Widgets
#Importing Widgets to allow for the changing of variables on the fly as questions are asked.
import ipywidgets as widgets 


# Input Data 

## Geographical Data

This file below is from the Ethiopian files available from the database on the following site:
https://data.humdata.org/dataset/ethiopia-population-data-_-admin-level-0-3

This site has both topography and the Level 0-3 admin data on the population levels per county/province. Vodacom only provided at Admin 1 (Provincial level) for this RFQ; but we are able to go more in depth to try and work out if there is any additional information we can use to strengthen our position.

The following section will pull that information from my Github; so that you don't have to fetch it yourself. Then load it as a JSON file into the DB, which you can see are Polygon type files with GPS coordinates which mark out the different layers/levels in the country. 

If needed you can pull the information from the GeoJSON files as well, but i did include the boundaries data as a dataframe too. 
Example: counties3["features"][0]['properties'] #Just a sample on how to pull out specific information from the Counties json Files.

## Admin Level Data

As mentioned above; there is both Geo and Admin data; this information matches the information against some paramater; since the file is nicely structured according to standards we will stick to the humanitarian markings. 

Below i read the information from different levels into the variables for Admin1-3 so that we are able to use them to draw choropleth maps of the country. 

Once read into memory; it is possible to find matches against the specific parameters in both the GeoJSON and the Admin files. So i run a few sample commands to view what the data looks like. 

## Vodacom Sites Data
Vodacom has provided the Ethiopia site numbers, and the expected by year and by type, although the Vodacom breaks it down by height and rooftop; this might not be necessary from our point of view and should not impact the way we calculate this. 

For this we will need to figure out how to define rural/urban and so forth.

In [2]:
#*Geographical Data*
#Pull the Data I stored in my Github account for the analysis.

with urlopen('https://github.com/johnmeye/Ethiopia/raw/master/Ethiopia_JSON/eth_admbnda_adm1_csa_bofed_20190827.json') as response:
    counties1 = json.load(response)
    
with urlopen('https://github.com/johnmeye/Ethiopia/raw/master/Ethiopia_JSON/eth_admbnda_adm2_csa_bofed_20190827.json') as response:
    counties2 = json.load(response)
    
with urlopen('https://github.com/johnmeye/Ethiopia/raw/master/Ethiopia_JSON/eth_admbnda_adm3_csa_bofed_20190827.json') as response:
    counties3 = json.load(response)

#*Feature Data* 
#This data is available in the JSON files but its easier to manage from a tableset so i have pulled this below as well.
#Feature data is area, coordinates and naming conventions of each province/suburb/district in ethiopia.
    
Boundaries_Data1 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/eth_adminboundaries_tabulardata.xlsx",
                  sheet_name='Admin1')

Boundaries_Data2 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/eth_adminboundaries_tabulardata.xlsx",
                    sheet_name='Admin2')

Boundaries_Data3 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/eth_adminboundaries_tabulardata.xlsx",
                    sheet_name='Admin3')


#*Admin Level Data*

Admin1 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/ethiopia-population-data-_-admin-level-0-3.xlsx",
                   dtype={"admin1Pcode": str},
                   skiprows=[1],
                   sheet_name='Admin1')

Admin2 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/ethiopia-population-data-_-admin-level-0-3.xlsx",
                   dtype={"admin1Pcode": str},
                   skiprows=[1],
                   sheet_name='Admin2')

Admin3 = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/ethiopia-population-data-_-admin-level-0-3.xlsx",
                   dtype={"admin1Pcode": str},
                   skiprows=[1],
                   sheet_name='Admin3')


### Site information input

The input file from Vodacom gives the sites by province and by year for the install. 

This information is stored in the Dataframe **TotalSites** which includes *Total by year* and *Total by Province* which looks as follows:

In [35]:
#*Vodacom Data*
# this information was supplied by Vodacom and contains the expected sites per region.
Sites = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/TX%20BoQ%20v5-Internal.xlsx",
                   sheet_name='Site_Numbers')
Sites_New = pd.read_excel("https://github.com/johnmeye/Ethiopia/raw/master/TX%20BoQ%20v5-Internal.xlsx",
                   sheet_name='Site_Numbers_New')

Sites.rename(columns = {'Location':'admin1Name_en'}, inplace = True)
Sites_New.rename(columns = {'Location':'admin1Name_en'}, inplace = True)

Site_Year = Sites_New.groupby(['admin1Name_en']).sum()

TotalSites = pd.merge(Site_Year, Boundaries_Data1[['admin1Name_en', 'Shape_Area']],how='left', on=['admin1Name_en']) #Site combined with the geodata. Sizing found early.
TotalSites.rename(columns = {'Shape_Area':'Shape_Area_Admin1'}, inplace = True)

TotalSites.loc['Year_Total']= TotalSites.sum(numeric_only=True, axis=0)
TotalSites.loc[:,'Total']= TotalSites.loc[:,'Year1':'Year5'].sum(numeric_only=True, axis=1)
TotalSites


,admin1Name_en,Year1,Year2,Year3,Year4,Year5,Total,Shape_Area_Admin1
0,Addis Ababa,416.180998,30.0,0.0,0.0,142.480596,588.661594,0.044369
1,Afar,50.000000,3.0,4.0,37.0,46.000000,140.000000,7.879581
2,Amhara,338.000000,562.0,430.0,141.0,157.000000,1628.000000,12.901454
3,Benishangul Gumz,19.000000,1.0,2.0,19.0,32.000000,73.000000,4.149305
4,Dire Dawa,46.000000,29.0,0.0,0.0,26.000000,101.000000,0.086757
5,Gambela,11.000000,3.0,4.0,8.0,11.000000,37.000000,2.560910
6,Harari,23.000000,27.0,0.0,0.0,15.000000,65.000000,0.030512
7,Oromia,258.000000,532.0,594.0,542.0,607.000000,2533.000000,26.474660
8,SNNP,118.000000,116.0,218.0,234.0,195.000000,881.000000,8.928528
9,Somali,98.000000,91.0,35.0,65.0,89.000000,378.000000,25.466707


In [38]:
Sites_New

,Position,Type,admin1Name_en,Year1,Year2,Year3,Year4,Year5,Total
0,RT,Rooftop,Addis Ababa,76.000000,NaN,NaN,NaN,24.000000,100.000000
1,GF,25m,Addis Ababa,250.000000,5.0,NaN,NaN,100.000000,355.000000
2,GF,30m,Addis Ababa,NaN,24.0,NaN,NaN,18.000000,42.000000
3,GF,40m,Addis Ababa,NaN,1.0,NaN,NaN,NaN,1.000000
4,GF,50m,Addis Ababa,90.180998,0.0,0.0,0.0,0.480596,90.661594
5,RT,Rooftop,Afar,NaN,NaN,NaN,NaN,NaN,0.000000
6,GF,25m,Afar,1.000000,NaN,NaN,1.0,1.000000,3.000000
7,GF,30m,Afar,5.000000,NaN,NaN,NaN,3.000000,8.000000
8,GF,40m,Afar,3.000000,NaN,NaN,5.0,6.000000,14.000000
9,GF,50m,Afar,41.000000,3.0,4.0,31.0,36.000000,115.000000


### Merging the Provincial Data and the Site Data given by Vodacom

This section briefly merges the information provided with the Site information. This allows a large single table which will then have all kinds of information about population; number of sites; land area and so forth.


In [20]:
#SiteTable is now created to hold the Sites and the Admin3 data together.
SiteTable = pd.merge(Admin3, TotalSites, how='outer', on=['admin1Name_en']) #Site by year
SiteTable.replace(to_replace=0, value=np.nan, inplace=True)

#Dropping a lot of information from Admin3 here to speed things up; we can return later if need be.
SiteTable.drop(SiteTable.iloc[:, 9:], inplace = True, axis = 1) 
SiteTable.drop(SiteTable.iloc[:, 6:8], inplace = True, axis = 1) 

#The Full Table now adds the Shape Area to this Table to allow us to know the total space available for the specific area.
Fulltable = pd.merge(SiteTable, Boundaries_Data3[['admin3Pcode', 'Shape_Area']],how='left', on=['admin3Pcode']) #Site combined with the geodata. Sizing found early.
Fulltable = Fulltable.dropna(subset=['admin3Name_en'])

# Some Shape_Area's are ' NaN'which is a bit of a problem when calculation of density; so i'm filling in these empty cells.
Fulltable['Shape_Area'] = Fulltable['Shape_Area'].fillna(0)

#Based on information from Wikipedia this is the total size of Ethiopia. The Shape Area is a percentage of the total areas.
#I originally used the following, but i think that the total areas on Wiki and This Shape Area don't add up to 100% because 
#The Shape area at Admin3 doesn't include the large bodies of water in the country which are not part of geographic boundaries.
#I'm going to assume that geo boundaries add up to 100%
    #Ethiopia_Shape_Area = Fulltable['Shape_Area'].sum()
    #Unit_Area = Ethiopia_Area / Ethiopia_Shape_Area

Ethiopia_Area = 1104300
Unit_Area = Ethiopia_Area / 100 

Fulltable['Area_Km'] = Fulltable['Shape_Area'].apply(lambda x: x*Unit_Area)
Fulltable.rename(columns = {'Total Population':'Total_Population'}, inplace = True)


def POPDENSITY(Population,Area,AreaName):
    try:
        Density=Population/Area
        return Density
    except ZeroDivisionError:
        print("Population Lost:", AreaName)
        return 0

Fulltable['Population_Density'] = Fulltable.apply(lambda x: POPDENSITY(x.Total_Population, x.Area_Km, x.admin3Name_en), axis=1)

Fulltable.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 983 entries, 0 to 982
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   admin3Name_en       983 non-null    object 
 1   admin3Pcode         983 non-null    object 
 2   admin2Name_en       983 non-null    object 
 3   admin2Pcode         983 non-null    object 
 4   admin1Name_en       983 non-null    object 
 5   admin1Pcode         983 non-null    object 
 6   Total_Population    982 non-null    float64
 7   Shape_Area          983 non-null    float64
 8   Area_Km             983 non-null    float64
 9   Population_Density  982 non-null    float64
dtypes: float64(4), object(6)
memory usage: 84.5+ KB


In [25]:
#This is a function which will allow us to plot a Choromap. it needs the following:
#DF          = Data frame that we want to plot.
#JSON_ON     = What are you using to build the geographic information ex:counties3
#Location_ON = What do you want to match on to link the Location and JSON information ex:'admin3Pcode'
#Feature_ON  = This is how you match on the JSON ex:'properties.ADM3_PCODE' this code and the Location_ON code must match to draw correctly.
#Color_ON    = This is a colour gradient on some aspect; ex:'Total_Population'
#RANGE_L     = Colour lower range bound (ie the BLUE)
#RANGE_U     = Colour upper range bound (ie the RED)
#TITLE       = Title and the saved name of the file: ex:'Total Population of a Region'
#HNAME       = The hover data which is shown when moused over: ie.'admin3Name_en'
#HDATA       = This is additional information that you want from the dataframe when hovered ie:["admin1Name_en", "Total_Population", "Area_Km"]
#
#
#The function then creates and saves an offline map in the Plotly folder so that we don't have to recreate this map every time. 

def CHOROMAP(DF,JSON,LOCATION,FEATUREKEY,COLOR,RANGE_L,RANGE_U,TITLE,HNAME,HDATA):
    Figure = px.choropleth_mapbox(DF, 
                        geojson=JSON, 
                        locations=LOCATION, featureidkey=FEATUREKEY,
                        color=COLOR,
                        color_continuous_scale="portland",
                        range_color=(RANGE_L, RANGE_U),
                        mapbox_style="carto-positron",
                        zoom=6, center = {"lat": 9, "lon": 39},
                        opacity=0.5,
                        hover_name=  HNAME,
                        hover_data=  HDATA,
                        labels={
                            'Population_Density':'Population Density',
                            'Total_Population':'Total Population',
                            'admin1Name_en':'Province',
                            'Area_Km':'Square Kilometer'
                        },
                        title = TITLE
                        )
    
    Figure.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    title={
            'text': TITLE,
            'y':0.9,
            'x':0.4,
            'xanchor': 'center',
            'yanchor': 'top'}
    )
    
    plotly.offline.plot(Figure, filename=("C:\\Users\\johnmeye\\Documents\\GitHub\\Ethiopia\\Plotly\\"+TITLE+".html"), auto_open=True)
    
    return Figure

In [24]:
JSON_ON     = counties3
Location_ON = 'admin3Pcode'
Feature_ON  = 'properties.ADM3_PCODE'
Color_ON    = 'Total_Population'
RANGE_L     = 25000
RANGE_U     = 300000
TITLE       = 'Total Population of a Region'
HNAME       = 'admin3Name_en'
HDATA       = ["admin1Name_en", "Total_Population", "Population_Density", "Area_Km"]

RANGE_L  = 25000
RANGE_U  = 300000
Color_ON = 'Total_Population'
Pop_Map  = CHOROMAP(Fulltable, JSON_ON, Location_ON, Feature_ON, Color_ON, RANGE_L, RANGE_U, TITLE, HNAME, HDATA)

RANGE_L  = 50
RANGE_U  = 500
Color_ON = 'Population_Density'
TITLE    = 'Population Density of a Region'
Dens_Map = CHOROMAP(Fulltable, JSON_ON, Location_ON, Feature_ON, Color_ON, RANGE_L, RANGE_U, TITLE, HNAME, HDATA)
